In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MM']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('multiple myeloma', 3699),
 ('malignant mesothelioma', 293),
 ('malignant melanoma', 186),
 ('metanephric mesenchyme', 96),
 ('metastatic melanoma', 54),
 ('molecular mechanics', 52),
 ('mismatch', 39),
 ('melastoma malabathricum linn', 35),
 ('minimal medium', 24),
 ('moneymaker', 17),
 ('mycophenolate mofetil', 12),
 ('masseter muscle', 12),
 ('medical management', 11),
 ('muscle mass', 11),
 ('maintenance medium', 11),
 ('macaca mulatta', 10),
 ('mirror movements', 9),
 ('macromolecular', 9),
 ('multiple micronutrient', 8),
 ('mindfulness meditation', 7),
 ('mixed micelles', 7),
 ('middle marsh', 6),
 ('maternal mortality', 6),
 ('menstrual migraine', 6),
 ('miyoshi myopathy', 6),
 ('middle molecules', 5),
 ('medial meniscectomy', 5),
 ('major minimal', 5),
 ('mild malaria', 5),
 ('melastoma malabathricum', 5),
 ('mitochondrial myopathy', 5),
 ('molecular micelles', 5),
 ('mandibular movements', 5),
 ('molecular mass', 5),
 ('marimastat', 5),
 ('melamine', 5),
 ('main genotypic ef

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-02 02:24:28] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'m mode': 'ungrounded',
  'macaca mulatta': 'MESH:D008253',
  'macromolecular': 'ungrounded',
  'magnetosome membrane': 'GO:GO:0110146',
  'main genotypic effect model': 'ungrounded',
  'maintenance medium': 'ungrounded',
  'major minimal': 'ungrounded',
  'malignant melanoma': 'MESH:D008545',
  'malignant mesothelioma': 'EFO:1000355',
  'malignant myeloma': 'MESH:D009101',
  'mandibular movements': 'ungrounded',
  'marimastat': 'CHEBI:CHEBI:50662',
  'marine metabolites': 'CHEBI:CHEBI:76507',
  'masseter muscle': 'MESH:D008406',
  'maternal mortality': 'MESH:D008428',
  'mathematics motivation': 'ungrounded',
  'matrigel microspheres': 'ungrounded',
  'matrix m': 'MESH:C000625666',
  'mecamylamine': 'CHEBI:CHEBI:6706',
  'medial mammillary nucleus': 'ungrounded',
  'medial meniscectomy': 'MESH:D000074403',
  'medial meniscus': 'MESH:D000072600',
  'medical management': 'ungrounded',
  'mediterranean meal': 'ungrounded',
  'melamine': 'CHEBI:CHEBI:27915',
  'melastoma malabathricum':

In [7]:
grounding_map, names, pos_labels = [{'m mode': 'ungrounded',
  'macaca mulatta': 'MESH:D008253',
  'macromolecular': 'ungrounded',
  'magnetosome membrane': 'GO:GO:0110146',
  'main genotypic effect model': 'ungrounded',
  'maintenance medium': 'ungrounded',
  'major minimal': 'ungrounded',
  'malignant melanoma': 'MESH:D008545',
  'malignant mesothelioma': 'EFO:1000355',
  'malignant myeloma': 'MESH:D009101',
  'mandibular movements': 'ungrounded',
  'marimastat': 'CHEBI:CHEBI:50662',
  'marine metabolites': 'CHEBI:CHEBI:76507',
  'masseter muscle': 'MESH:D008406',
  'maternal mortality': 'MESH:D008428',
  'mathematics motivation': 'ungrounded',
  'matrigel microspheres': 'ungrounded',
  'matrix m': 'MESH:C000625666',
  'mecamylamine': 'CHEBI:CHEBI:6706',
  'medial mammillary nucleus': 'ungrounded',
  'medial meniscectomy': 'MESH:D000074403',
  'medial meniscus': 'MESH:D000072600',
  'medical management': 'ungrounded',
  'mediterranean meal': 'ungrounded',
  'melamine': 'CHEBI:CHEBI:27915',
  'melastoma malabathricum': 'TAXONOMY:128733',
  'melastoma malabathricum linn': 'TAXONOMY:128733',
  'meningomyelocele': 'MESH:D008591',
  'menstrual migraine': 'MESH:D008881',
  'mesangial matrix': 'ungrounded',
  'metabolic memory': 'ungrounded',
  'metal mixture': 'ungrounded',
  'metanephric mesenchyme': 'metanephric_mesenchyme',
  'metastatic melanoma': 'EFO:0002617',
  'methadone maintained': 'ungrounded',
  'methylmercury': 'CHEBI:CHEBI:30785',
  'middle marsh': 'ungrounded',
  'middle molecules': 'ungrounded',
  'mild malaria': 'MESH:D008288',
  'mindfulness meditation': 'MESH:D019122',
  'mineral matter': 'ungrounded',
  'mineralisation medium': 'ungrounded',
  'minimal medium': 'ungrounded',
  'mirror movements': 'HP:HP:0001335',
  'mismatch': 'ungrounded',
  'mismatch oligonucleotides': 'ungrounded',
  'mithramycin': 'CHEBI:CHEBI:31856',
  'mitochondrial membrane': 'GO:GO:0031966',
  'mitochondrial myopathy': 'MESH:D017240',
  'mitomycin c': 'CHEBI:CHEBI:27504',
  'mixed micelles': 'ungrounded',
  'miyoshi myopathy': 'MESH:C537480',
  'mm': 'ungrounded',
  'mobilization macromolecules': 'ungrounded',
  'mode mic': 'ungrounded',
  'molecular mass': 'ungrounded',
  'molecular mechanics': 'ungrounded',
  'molecular micelles': 'ungrounded',
  'moneymaker': 'ungrounded',
  'monocytes macrophages': 'ungrounded',
  'monodora myristica': 'MESH:D026323',
  'morris water maze': 'ungrounded',
  'multimodal': 'ungrounded',
  'multiple micronutrient': 'CHEBI:CHEBI:27027',
  'multiple myeloma': 'MESH:D009101',
  'muscle mass': 'ungrounded',
  'muscularis mucosa': 'ungrounded',
  'mycophenolate mofetil': 'CHEBI:CHEBI:8764',
  'myeloma multiplex': 'ungrounded'},
 {'MESH:D008253': 'Macaca mulatta',
  'GO:GO:0110146': 'magnetosome membrane',
  'MESH:D008545': 'Melanoma',
  'EFO:1000355': 'Malignant Mesothelioma',
  'MESH:D009101': 'Multiple Myeloma',
  'CHEBI:CHEBI:50662': 'marimastat',
  'CHEBI:CHEBI:76507': 'marine metabolite',
  'MESH:D008406': 'Masseter Muscle',
  'MESH:D008428': 'Maternal Mortality',
  'MESH:C000625666': 'Matrix-M',
  'CHEBI:CHEBI:6706': 'Mecamylamine',
  'MESH:D000074403': 'Meniscectomy',
  'MESH:D000072600': 'Meniscus',
  'CHEBI:CHEBI:27915': 'melamine',
  'TAXONOMY:128733': 'Melastoma malabathricum L.',
  'MESH:D008591': 'Meningomyelocele',
  'MESH:D008881': 'Migraine Disorders',
  'metanephric_mesenchyme': 'metanephric_mesenchyme',
  'EFO:0002617': 'metastatic melanoma',
  'CHEBI:CHEBI:30785': 'methylmercury(.)',
  'MESH:D008288': 'Malaria',
  'MESH:D019122': 'Meditation',
  'HP:HP:0001335': 'Bimanual synkinesia',
  'CHEBI:CHEBI:31856': 'mithramycin',
  'GO:GO:0031966': 'mitochondrial membrane',
  'MESH:D017240': 'Mitochondrial Myopathies',
  'CHEBI:CHEBI:27504': 'mitomycin C',
  'MESH:C537480': 'Miyoshi myopathy',
  'MESH:D026323': 'Myristica',
  'CHEBI:CHEBI:27027': 'micronutrient',
  'CHEBI:CHEBI:8764': 'mycophenolate mofetil'},
 ['EFO:0002617',
  'EFO:1000355',
  'MESH:D008545',
  'MESH:D009101',
  'TAXONOMY:128733']]

In [8]:
excluded_longforms = []

In [9]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [10]:
additional_entities = {}

In [11]:
unambiguous_agent_texts = {}

In [12]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [19]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [20]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [21]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [22]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

[('HGNC:7321', ['Musculin', 'musculin'])]

In [23]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [13]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-22 00:48:49] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-22 00:51:20] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9407089297583602 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [14]:
classifier.stats

{'label_distribution': {'MESH:D009101': 2548,
  'MESH:D008545': 127,
  'ungrounded': 177,
  'CHEBI:CHEBI:30785': 4,
  'CHEBI:CHEBI:31856': 3,
  'EFO:0002617': 39,
  'EFO:1000355': 200,
  'CHEBI:CHEBI:8764': 8,
  'MESH:C537480': 5,
  'MESH:D008428': 4,
  'GO:GO:0031966': 3,
  'metanephric_mesenchyme': 76,
  'MESH:D008881': 4,
  'MESH:D008288': 2,
  'HP:HP:0001335': 7,
  'CHEBI:CHEBI:27027': 6,
  'MESH:D008591': 2,
  'CHEBI:CHEBI:50662': 3,
  'MESH:D017240': 4,
  'CHEBI:CHEBI:27915': 2,
  'CHEBI:CHEBI:6706': 3,
  'MESH:D019122': 4,
  'TAXONOMY:128733': 3,
  'MESH:D008406': 7,
  'CHEBI:CHEBI:76507': 1,
  'MESH:D026323': 1,
  'MESH:C000625666': 2,
  'MESH:D000074403': 1,
  'GO:GO:0110146': 1,
  'MESH:D008253': 1,
  'CHEBI:CHEBI:27504': 1,
  'MESH:D000072600': 3},
 'f1': {'mean': 0.940709, 'std': 0.006326},
 'precision': {'mean': 0.931624, 'std': 0.006413},
 'recall': {'mean': 0.957491, 'std': 0.005879},
 'MESH:D009101': {'f1': {'mean': 0.965588, 'std': 0.00286},
  'pr': {'mean': 0.996466, 

In [15]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [16]:
disamb.dump(model_name, results_path)

In [17]:
print(disamb.info())

Disambiguation model for MM

Produces the disambiguations:
	Bimanual synkinesia	HP:HP:0001335
	Macaca mulatta	MESH:D008253
	Malaria	MESH:D008288
	Malignant Mesothelioma*	EFO:1000355
	Masseter Muscle	MESH:D008406
	Maternal Mortality	MESH:D008428
	Matrix-M	MESH:C000625666
	Mecamylamine	CHEBI:CHEBI:6706
	Meditation	MESH:D019122
	Melanoma*	MESH:D008545
	Melastoma malabathricum L.*	TAXONOMY:128733
	Meningomyelocele	MESH:D008591
	Meniscectomy	MESH:D000074403
	Meniscus	MESH:D000072600
	Migraine Disorders	MESH:D008881
	Mitochondrial Myopathies	MESH:D017240
	Miyoshi myopathy	MESH:C537480
	Multiple Myeloma*	MESH:D009101
	Myristica	MESH:D026323
	magnetosome membrane	GO:GO:0110146
	marimastat	CHEBI:CHEBI:50662
	marine metabolite	CHEBI:CHEBI:76507
	melamine	CHEBI:CHEBI:27915
	metanephric_mesenchyme	metanephric_mesenchyme
	metastatic melanoma*	EFO:0002617
	methylmercury(.)	CHEBI:CHEBI:30785
	micronutrient	CHEBI:CHEBI:27027
	mithramycin	CHEBI:CHEBI:31856
	mitochondrial membrane	GO:GO:0031966
	mitomyc

In [18]:
model_to_s3(disamb)